In [4]:
import pandas as pd
from google.colab import drive

# 1. Montage Drive
drive.mount('/content/drive')

# 1. Chargement des fichiers
# Remplace les chemins par tes chemins réels (ex: /content/drive/...)
df_links = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tmdb_allfilms.csv')
df_business = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projet_thinking/movies_ALL_20M.csv')

# 2. Nettoyage préventif
# On s'assure que les colonnes de jointure n'ont pas de valeurs nulles et sont du même type
df_links = df_links.dropna(subset=['movieId', 'tmdbId'])
df_links['movieId'] = df_links['movieId'].astype(int)
df_links['tmdbId'] = df_links['tmdbId'].astype(int)


# 4. Deuxième fusion : Ajouter les données Business
# On fusionne le résultat précédent avec df_business sur 'tmdbId'
df_final = pd.merge(df_links, df_business, on='tmdbId', how='inner')

# 5. Suppression des colonnes dupliquées
# Si certains fichiers avaient des colonnes identiques (ex: 'title'),
# Pandas crée 'title_x' et 'title_y'. Ce code les nettoie :
df_final = df_final.loc[:, ~df_final.columns.duplicated()]

# Optionnel : Si tu as des colonnes comme 'title_x' et 'title_y' à cause du merge
if 'title_x' in df_final.columns and 'title_y' in df_final.columns:
    df_final['title'] = df_final['title_x']
    df_final = df_final.drop(['title_x', 'title_y'], axis=1)

# 6. Sauvegarde du résultat
df_final.to_csv('/content/drive/MyDrive/Colab Notebooks/projet_thinking/final_merged_movies.csv', index=False)

print(f"✅ Fusion réussie ! Taille du dataset final : {df_final.shape}")
print(df_final.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Fusion réussie ! Taille du dataset final : (26782, 18)
   movieId  imdbId  tmdbId                                        description  \
0        1  114709     862  Dans un monde où les jouets vivent leur vie qu...   
1        2  113497    8844  Lors d'une partie de Jumanji, un jeu très anci...   
2        3  113228   15602  Après le mariage de John et d'Ariel, Max se re...   
3        4  114885   31357  L'amitié de quatre femmes qui tentent de surmo...   
4        5  113041   11862  George Banks et sa petite famille sont de reto...   

                                          poster_url  note_tmdb  \
0  https://image.tmdb.org/t/p/w500/4cWLRhub0yY9VJ...      7.971   
1  https://image.tmdb.org/t/p/w500/nLXYV4WmYUF4i8...      7.244   
2  https://image.tmdb.org/t/p/w500/zRLjlOwgJtyZCN...      6.482   
3  https://image.tmdb.org/t/p/w500/mIRqwlKWmzdCpn...      6

In [ ]:
import pandas as pd
import requests
import time
import os
from google.colab import drive

# 1. Montage Drive
drive.mount('/content/drive')

# --- CONFIGURATION ---
API_KEY = "5397e620fd3a7e92001e39c5bd3367ab"
INPUT_CSV = "/content/drive/MyDrive/Colab Notebooks/links.csv"
OUTPUT_CSV = "/content/drive/MyDrive/Colab Notebooks/projet_thinking/movies_ALL_20M.csv"
BATCH_SIZE = 5000

def get_detailed_data(tmdb_id):
    url = f"https://api.themoviedb.org/3/movie/{tmdb_id}?api_key={API_KEY}&language=en-US"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            # Extraction propre des genres et studios (format texte séparé par des pipes)
            genres = "|".join([g['name'] for g in data.get('genres', [])])
            studios = "|".join([c['name'] for c in data.get('production_companies', [])])

            return {
                'tmdbId': tmdb_id,
                'title': data.get('title'),
                'budget': data.get('budget', 0),
                'revenue': data.get('revenue', 0),
                'popularity': data.get('popularity', 0),
                'vote_average': data.get('vote_average', 0),
                'vote_count': data.get('vote_count', 0),
                'genres': genres,
                'production_companies': studios,
                'runtime': data.get('runtime', 0),
                'release_date': data.get('release_date', ''),
                'status': data.get('status', '')
            }
    except Exception as e:
        return None
    return None

# 2. Chargement et Check de progression
df_links = pd.read_csv(INPUT_CSV).dropna(subset=['tmdbId'])
df_links['tmdbId'] = df_links['tmdbId'].astype(int)

# Si le fichier existe déjà, on regarde où on s'est arrêté
if os.path.exists(OUTPUT_CSV):
    existing_df = pd.read_csv(OUTPUT_CSV)
    processed_ids = set(existing_df['tmdbId'].unique())
    df_to_process = df_links[~df_links['tmdbId'].isin(processed_ids)]
    print(f"🔄 Reprise : {len(processed_ids)} déjà faits, reste {len(df_to_process)} à traiter.")
else:
    df_to_process = df_links
    print(f"🚀 Premier lancement pour {len(df_to_process)} films.")

enriched_data = []

# 3. Boucle de scraping
for index, row in df_to_process.iterrows():
    info = get_detailed_data(row['tmdbId'])

    if info:
        # Calcul du ROI ici pour gagner du temps au Dashboard
        b = info['budget']
        r = info['revenue']
        info['roi'] = round((r - b) / b, 2) if b > 0 else 0
        enriched_data.append(info)

    # Sauvegarde par batch
    if len(enriched_data) >= BATCH_SIZE:
        temp_df = pd.DataFrame(enriched_data)
        if not os.path.isfile(OUTPUT_CSV):
            temp_df.to_csv(OUTPUT_CSV, index=False)
        else:
            temp_df.to_csv(OUTPUT_CSV, mode='a', header=False, index=False)

        print(f"💾 Batch sauvé. Total traité : {len(enriched_data)} dans ce cycle.")
        enriched_data = [] # Reset memoire

    time.sleep(0.05) # Respecter les limites TMDB

# Sauvegarde finale du reste
if enriched_data:
    pd.DataFrame(enriched_data).to_csv(OUTPUT_CSV, mode='a', header=False, index=False)

print("✅ Scraping complet terminé !")

Mounted at /content/drive
🚀 Premier lancement pour 27026 films.
💾 Batch sauvé. Total traité : 5000 dans ce cycle.
💾 Batch sauvé. Total traité : 5000 dans ce cycle.
💾 Batch sauvé. Total traité : 5000 dans ce cycle.
💾 Batch sauvé. Total traité : 5000 dans ce cycle.
💾 Batch sauvé. Total traité : 5000 dans ce cycle.
✅ Scraping complet terminé !


In [ ]:
import pandas as pd
from google.colab import drive

# 1. Montage du Drive
drive.mount('/content/drive')

# 2. Chargement des fichiers
print("⌛ Chargement des fichiers...")
movis = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/movies.csv")
links = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tmdb_allfilms.csv")
movisbusiness = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/projet_thinking/movies_business_enriched.csv")

# 3. Première fusion : Movies + Links (via movieId)
# On récupère le tmdbId pour pouvoir faire le lien avec le fichier business
df_combined = pd.merge(movis, links, on='movieId', how='inner')

# 4. Deuxième fusion : Résultat + MoviesBusiness (via tmdbId)
# On utilise 'inner' pour ne garder que les films qui ont des données business
df_final = pd.merge(df_combined, movisbusiness, on='tmdbId', how='inner')

# 5. Nettoyage (Optionnel : supprimer les colonnes en double si elles existent)
# Si 'title' existe dans les deux fichiers, pandas crée 'title_x' et 'title_y'
if 'title_x' in df_final.columns:
    df_final['title'] = df_final['title_x']
    df_final = df_final.drop(['title_x', 'title_y'], axis=1)

# 6. Aperçu et Sauvegarde
print(f"✅ Fusion terminée ! Nombre total de films enrichis : {len(df_final)}")
print(df_final.head())



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⌛ Chargement des fichiers...
✅ Fusion terminée ! Nombre total de films enrichis : 26783
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  imdbId   tmdbId  \
0  Adventure|Animation|Children|Comedy|Fantasy  114709    862.0   
1                   Adventure|Children|Fantasy  113497   8844.0   
2                               Comedy|Romance  113228  15602.0   
3                         Comedy|Drama|Romance  114885  31357.0   
4                                       Comedy  113041  11862.0   

                                         description  \
0  Dans un monde où les

In [ ]:
# Sauvegarde du dataset final pour le projet
df_final.to_csv("/content/drive/MyDrive/Colab Notebooks/projet_thinking/final_enriched_movies.csv", index=False)